In [2]:
import numpy as np
import PIL.Image as image
import os
import matplotlib.pyplot as plt
import cv2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets,models,transforms
import time
import copy

In [4]:
data_dir_train='/home/user/Documents/Niranjana/Plant Disease Model/New Plant Diseases Dataset(Augmented)/train'
data_dir_test='/home/user/Documents/Niranjana/Plant Disease Model/New Plant Diseases Dataset(Augmented)/valid'

In [5]:
import pathlib
data_dir_train=pathlib.Path(data_dir_train)
data_dir_train

PosixPath('/home/user/Documents/Niranjana/Plant Disease Model/New Plant Diseases Dataset(Augmented)/train')

In [6]:
classes = [subdir.name for subdir in data_dir_train.glob('*') if subdir.is_dir()]



In [7]:
classes

['Apple___Cedar_apple_rust',
 'Grape___Black_rot',
 'Corn_(maize)___healthy',
 'Potato___healthy',
 'Grape___Esca_(Black_Measles)',
 'Tomato___Late_blight',
 'Peach___Bacterial_spot',
 'Tomato___Target_Spot',
 'Peach___healthy',
 'Tomato___Leaf_Mold',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Potato___Late_blight',
 'Grape___healthy',
 'Raspberry___healthy',
 'Tomato___Tomato_mosaic_virus',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Tomato___healthy',
 'Pepper,_bell___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Corn_(maize)___Common_rust_',
 'Tomato___Bacterial_spot',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Early_blight',
 'Soybean___healthy',
 'Apple___healthy',
 'Cherry_(including_sour)___healthy',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Pepper,_bell___Bacterial_spot',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Apple___Apple_scab',
 'Blueberry___healthy',
 'Orange___Haunglongbing_(Ci

In [8]:
len(classes)

38

In [9]:
leaves_labels={}
for index,class_names in enumerate(classes):
    leaves_labels[class_names]=index
    # leaves_labels[class_names]=i

In [10]:
leaves={}
for class_name in classes:
    class_path=data_dir_train/class_name
    leaves[class_name]=list(class_path.glob('*'))

y_train=[]
for plant_name,path in leaves.items():
    for image_path in path:
        y_train.append(leaves_labels[plant_name])


In [11]:
data_dir_test=pathlib.Path(data_dir_test)


In [12]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((244, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [13]:
from torchvision.datasets import ImageFolder
train_dataset = ImageFolder(root=data_dir_train, transform=transform)
test_dataset = ImageFolder(root=data_dir_test, transform=transform)

In [14]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
from PIL import Image as image

In [16]:
train_dataset[0][0].shape

torch.Size([3, 244, 224])

In [17]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

Using cache found in /home/user/.cache/torch/hub/pytorch_vision_v0.10.0
/home/user/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [21]:
num_classes=len(classes)
# Modify the classifier layers for the new number of classes
for param in model.parameters():
    param.requires_grad=False
model.features[10].requires_grad=True
model.features[12].requires_grad=True
model.classifier=nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(512, +num_classes, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1, 1))

    )

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define your criterion (loss function)
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

# Evaluation
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

test_accuracy = correct_predictions / total_predictions
print("Test Accuracy: {:.2%}".format(test_accuracy))  


Epoch [1/100], Loss: 0.2252227872330102
Epoch [2/100], Loss: 0.09178098411460028
Epoch [3/100], Loss: 0.07703334246409592
Epoch [4/100], Loss: 0.06895379091771071
Epoch [5/100], Loss: 0.0649038566650209
Epoch [6/100], Loss: 0.06269230637748578
Epoch [7/100], Loss: 0.05949066794613696
Epoch [8/100], Loss: 0.05942800963150987
Epoch [9/100], Loss: 0.05687389822352756
Epoch [10/100], Loss: 0.05589735012264643
Epoch [11/100], Loss: 0.05414907380646687
Epoch [12/100], Loss: 0.05347419818456803
Epoch [13/100], Loss: 0.05327563300553012
Epoch [14/100], Loss: 0.05313536391964467
Epoch [15/100], Loss: 0.052553674072916966
Epoch [16/100], Loss: 0.05134259395844868
Epoch [17/100], Loss: 0.05226388434040785
Epoch [18/100], Loss: 0.05128176446391766
Epoch [19/100], Loss: 0.05121226596218467
Epoch [20/100], Loss: 0.049082376355476055
Epoch [21/100], Loss: 0.04781524166602235
Epoch [22/100], Loss: 0.049018025830786086
Epoch [23/100], Loss: 0.04936987569399897
Epoch [24/100], Loss: 0.04773625141558514


In [22]:
# Count the number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())

# Calculate the model size in bytes
model_size_bytes = total_params * 4  # Assuming 32-bit float (4 bytes) for each parameter

print("Model size in bytes: ", model_size_bytes)
size_kb = model_size_bytes / 1024
size_mb = size_kb / 1024

print("Model size in KB: ", size_kb)
print("Model size in MB: ", size_mb)

Model size in bytes:  3019672
Model size in KB:  2948.8984375
Model size in MB:  2.8797836303710938


In [24]:
#Save the trained model
torch.save(model.state_dict(), "model.pth")

In [25]:
scripted_model=torch.jit.script(model)

In [26]:
from torch.utils.mobile_optimizer import optimize_for_mobile
opt_model=optimize_for_mobile(scripted_model)

In [27]:
opt_model._save_for_lite_interpreter("mobile_model.pt")